In [24]:
import yaml
from pathlib import Path
import pandas as pd

In [25]:
# Transformation
# File path -> Metadata
def markdown_to_metadata(file_path):
    """Transform markdown to metadata file."""
    with file_path.open("r", encoding="utf-8") as file:
        content = file.read()
        if content.startswith("---"):
            parts = content.split("---", 2)
            if len(parts) > 1:
                metadata = yaml.safe_load(parts[1].strip())  # Parse YAML front matter
                content_without_front_matter = parts[2].strip()  # Content excluding front matter
                return {**metadata, "content": content_without_front_matter}
    return None

def folder_to_metadata(folder_path):
    """Analyze all markdown files in a given folder and return metadata."""
    if not folder_path.exists():
        raise FileNotFoundError(f"The folder {folder_path} does not exist.")

    return [
        {**markdown_metadata}
        for file_path in folder_path.rglob("*.md")
        if (markdown_metadata := markdown_to_metadata(file_path)) is not None
    ]

## Pricing related EIPs

In [26]:
folder_path = Path.cwd().parents[1] / "EIPs" / "EIPS" # Points to the EIPs Repostiory
df = pd.DataFrame(folder_to_metadata(folder_path))

# Keywords to search
keywords = ['cost', 'gas', 'price', 'pricing']

pricing_eips = df[
    (df['type'] == 'Standards Track') &  # Type is Standards Track
    (df['category'] == 'Core') &  # Category is Core
    (df['title'].str.contains('|'.join(keywords), case=False, na=False) |  # Title contains any of the keywords
     df['description'].str.contains('|'.join(keywords), case=False, na=False))  # Description contains any of the keywords
]
pricing_eips

,eip,title,description,author,discussions-to,status,type,category,created,requires,content,last-call-deadline,withdrawal-reason,updated
49,2565,ModExp Gas Cost,NaN,"Kelly Olson (@ineffectualproperty), Sean Gulle...",https://ethereum-magicians.org/t/big-integer-m...,Final,Standards Track,Core,2020-03-20,198,## Simple Summary\nDefines the gas cost of the...,NaN,NaN,NaN
65,7686,Linear EVM memory limits,Adjust memory limits and gas limits of sub-cal...,Vitalik Buterin (@vbuterin),https://ethereum-magicians.org/t/eip-7686-line...,Stagnant,Standards Track,Core,2024-04-15,NaN,## Abstract\n\nAdd a hard memory limit equal t...,NaN,NaN,NaN
86,1706,Disable SSTORE with gasleft lower than call st...,NaN,"Alex Forshtat <alex@tabookey.com>, Yoav Weiss ...",https://github.com/alex-forshtat-tbk/EIPs/issu...,Withdrawn,Standards Track,Core,2019-01-15,1283,## Simple Summary\nThe proposal that had been ...,NaN,The authors prefer [EIP-2200](./eip-2200.md),NaN
97,5022,Increase price of SSTORE from zero to non-zero...,NaN,Green (@greenlucid),https://ethereum-magicians.org/t/eip-proposal-...,Stagnant,Standards Track,Core,2022-04-20,NaN,### Abstract\n\nIncrease the price of the SSTO...,NaN,NaN,NaN
100,5,Gas Usage for `RETURN` and `CALL*`,NaN,Christian Reitwiessner <c@ethdev.com>,NaN,Final,Standards Track,Core,2015-11-22,NaN,### Abstract\n\nThis EIP makes it possible to ...,NaN,NaN,NaN
104,150,Gas cost changes for IO-heavy operations,NaN,Vitalik Buterin (@vbuterin),NaN,Final,Standards Track,Core,2016-09-24,NaN,### Meta reference\n\n[Tangerine Whistle](./ei...,NaN,NaN,NaN
153,7709,Read BLOCKHASH from storage and update cost,Read the `BLOCKHASH (0x40)` opcode from the EI...,"Vitalik Buterin (@vbuterin), Tomasz Stanczak (...",https://ethereum-magicians.org/t/eip-7709-read...,Draft,Standards Track,Core,2024-05-18,2935,## Abstract\n\nUpdate the `BLOCKHASH (0x40)` o...,NaN,NaN,NaN
192,3521,Reduce access list cost,NaN,Matt Garnett (@lightclient),https://ethereum-magicians.org/t/eip-3521-redu...,Stagnant,Standards Track,Core,2021-04-15,"2028, 2930",## Simple Summary\n\nReduce the cost of declar...,NaN,NaN,NaN
197,3860,Limit and meter initcode,Limit the maximum size of initcode to 49152 an...,"Martin Holst Swende (@holiman), Paweł Bylica (...",https://ethereum-magicians.org/t/eip-3860-limi...,Final,Standards Track,Core,2021-07-16,170,## Abstract\n\nWe extend [EIP-170](./eip-170.m...,NaN,NaN,NaN
212,1109,PRECOMPILEDCALL opcode (Remove CALL costs for ...,NaN,Jordi Baylina (@jbaylina),https://ethereum-magicians.org/t/eip-1109-remo...,Stagnant,Standards Track,Core,2018-05-22,NaN,## Simple Summary\n\nThis EIP creates a specif...,NaN,NaN,NaN


### Pricing related EIPs: Breakdown by status

In [27]:
pricing_eips['status'].value_counts(dropna=False)

status
Stagnant     20
Final        12
Draft        11
Withdrawn     4
Review        3
Name: count, dtype: int64

## EIPs not yet finalized

In [28]:
proposed_pricing_eips = pricing_eips[pricing_eips['status'] != 'Final']
proposed_pricing_eips

,eip,title,description,author,discussions-to,status,type,category,created,requires,content,last-call-deadline,withdrawal-reason,updated
65,7686,Linear EVM memory limits,Adjust memory limits and gas limits of sub-cal...,Vitalik Buterin (@vbuterin),https://ethereum-magicians.org/t/eip-7686-line...,Stagnant,Standards Track,Core,2024-04-15,NaN,## Abstract\n\nAdd a hard memory limit equal t...,NaN,NaN,NaN
86,1706,Disable SSTORE with gasleft lower than call st...,NaN,"Alex Forshtat <alex@tabookey.com>, Yoav Weiss ...",https://github.com/alex-forshtat-tbk/EIPs/issu...,Withdrawn,Standards Track,Core,2019-01-15,1283,## Simple Summary\nThe proposal that had been ...,NaN,The authors prefer [EIP-2200](./eip-2200.md),NaN
97,5022,Increase price of SSTORE from zero to non-zero...,NaN,Green (@greenlucid),https://ethereum-magicians.org/t/eip-proposal-...,Stagnant,Standards Track,Core,2022-04-20,NaN,### Abstract\n\nIncrease the price of the SSTO...,NaN,NaN,NaN
153,7709,Read BLOCKHASH from storage and update cost,Read the `BLOCKHASH (0x40)` opcode from the EI...,"Vitalik Buterin (@vbuterin), Tomasz Stanczak (...",https://ethereum-magicians.org/t/eip-7709-read...,Draft,Standards Track,Core,2024-05-18,2935,## Abstract\n\nUpdate the `BLOCKHASH (0x40)` o...,NaN,NaN,NaN
192,3521,Reduce access list cost,NaN,Matt Garnett (@lightclient),https://ethereum-magicians.org/t/eip-3521-redu...,Stagnant,Standards Track,Core,2021-04-15,"2028, 2930",## Simple Summary\n\nReduce the cost of declar...,NaN,NaN,NaN
212,1109,PRECOMPILEDCALL opcode (Remove CALL costs for ...,NaN,Jordi Baylina (@jbaylina),https://ethereum-magicians.org/t/eip-1109-remo...,Stagnant,Standards Track,Core,2018-05-22,NaN,## Simple Summary\n\nThis EIP creates a specif...,NaN,NaN,NaN
240,1285,Increase Gcallstipend gas in the CALL opcode,NaN,"Ben Kaufman <ben@daostack.io>, Adam Levi <adam...",https://ethereum-magicians.org/t/eip-1285-incr...,Stagnant,Standards Track,Core,2018-08-01,NaN,## Simple Summary\nIncrease the ``Gcallstipend...,NaN,NaN,NaN
270,7667,Raise gas costs of hash functions,Raise the gas costs of hash function opcodes a...,Vitalik Buterin (@vbuterin),https://ethereum-magicians.org/t/eip-7562-rais...,Draft,Standards Track,Core,2024-03-31,NaN,## Abstract\n\nRaise the gas costs of opcodes ...,NaN,NaN,NaN
272,4488,Transaction calldata gas cost reduction with t...,Greatly decreases the gas cost of transaction ...,"Vitalik Buterin (@vbuterin), Ansgar Dietrichs ...",https://ethereum-magicians.org/t/eip-4488-tran...,Stagnant,Standards Track,Core,2021-11-23,NaN,## Abstract\n\nDecrease transaction calldata g...,NaN,NaN,NaN
281,2045,Particle gas costs for EVM opcodes,NaN,"Casey Detrio (@cdetrio), Alex Beregszaszi (@axic)",https://ethereum-magicians.org/t/eip-2045-frac...,Stagnant,Standards Track,Core,2019-05-17,NaN,"## Abstract\nAccording to recent benchmarks, E...",NaN,NaN,NaN


## Meta - Helper function to generate markdown code

In [40]:
def df_to_markdown(df):
    # Map each row to the appropriate markdown format
    df = df.sort_values(by='eip') 
    markdown_rows = df.apply(lambda row: 
                             f"| [EIP-{row['eip']}: {row['title']}](https://eips.ethereum.org/EIPS/eip-{row['eip']}) |"
                             f" {row['description'] if pd.notna(row['description']) else '-'} |"
                             f"{row['status']} |"
                             f"{row['author']} |"
                             , axis=1)
    
    # Prepare the header and markdown table structure
    header = "| EIP | Description | Status | Author(s) |"
    separator = "|----------|----------|----------|----------|"
    
    # Combine everything into a full markdown table
    markdown_table = "\n".join([header, separator] + markdown_rows.tolist())
    
    return markdown_table

print(df_to_markdown(proposed_pricing_eips))

| EIP | Description | Status | Author(s) |
|----------|----------|----------|----------|
| [EIP-1087: Net gas metering for SSTORE operations](https://eips.ethereum.org/EIPS/eip-1087) | - |Stagnant |Nick Johnson (@arachnid) |
| [EIP-1109: PRECOMPILEDCALL opcode (Remove CALL costs for precompiled contracts)](https://eips.ethereum.org/EIPS/eip-1109) | - |Stagnant |Jordi Baylina (@jbaylina) |
| [EIP-1285: Increase Gcallstipend gas in the CALL opcode](https://eips.ethereum.org/EIPS/eip-1285) | - |Stagnant |Ben Kaufman <ben@daostack.io>, Adam Levi <adam@daostack.io> |
| [EIP-1380: Reduced gas cost for call to self](https://eips.ethereum.org/EIPS/eip-1380) | - |Stagnant |Alex Beregszaszi (@axic), Jacques Wagener (@jacqueswww) |
| [EIP-1706: Disable SSTORE with gasleft lower than call stipend](https://eips.ethereum.org/EIPS/eip-1706) | - |Withdrawn |Alex Forshtat <alex@tabookey.com>, Yoav Weiss <yoav@tabookey.com> |
| [EIP-1930: CALLs with strict gas semantic. Revert if not enough gas availabl